In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix
from collections import defaultdict

# === CONFIG ===
image_path = r"C:\Users\jotir\Downloads\train-2\images\3.jpg"  # Change to your image
label_path = r"C:\Users\jotir\Downloads\train-2\labels\3.txt"  # Ground truth label
model_path = "runs/detect/train11/weights/last.pt"  # YOLOv12 model
output_dir = 'runs/detect/eval'  # YOLOv12 output folder
iou_thresholds = np.arange(0.5, 1.0, 0.05)  # IoU thresholds for mAP50-95
class_names = ['image', 'text']  # class 0 = image, 1 = text

# === STEP 1: Load the YOLOv12 model ===
model = YOLO(model_path)

# === STEP 2: Run YOLOv12 Prediction ===
results = model(image_path, conf=0.8)  # Run inference with the given image and confidence threshold

# === STEP 3: Load Ground Truth YOLO Label ===
h, w, _ = cv2.imread(image_path).shape
gt_boxes = []
with open(label_path, 'r') as f:
    for line in f:
        cls, cx, cy, bw, bh = map(float, line.strip().split())
        x1 = (cx - bw/2) * w
        y1 = (cy - bh/2) * h
        x2 = (cx + bw/2) * w
        y2 = (cy + bh/2) * h
        gt_boxes.append([x1, y1, x2, y2, int(cls)])
gt_boxes = np.array(gt_boxes)

# === STEP 4: IoU Calculation ===
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# === STEP 5: Extract Prediction Boxes ===
# Get prediction boxes and labels
pred_boxes = []
for det in results:
    boxes = det.boxes.xyxy.cpu().numpy()  # Get the xyxy coordinates of predicted boxes
    confs = det.boxes.conf.cpu().numpy()  # Get the confidence scores
    clses = det.boxes.cls.cpu().numpy()  # Get the class ids
    for box, conf, cls in zip(boxes, confs, clses):
        x1, y1, x2, y2 = box
        pred_boxes.append([x1, y1, x2, y2, conf, int(cls)])
pred_boxes = np.array(pred_boxes)

# === STEP 6: Evaluate ===
TP = {cls: 0 for cls in range(len(class_names))}
FP = {cls: 0 for cls in range(len(class_names))}
FN = {cls: 0 for cls in range(len(class_names))}
IoUs = defaultdict(list)  # To store IoUs for each prediction
confidences = defaultdict(list)  # Store confidence for each class
matched = set()

# Calculate per-class TP, FP, FN and IoU
for pred in pred_boxes:
    matched_flag = False
    for i, gt in enumerate(gt_boxes):
        if i in matched: continue
        if pred[5] == gt[4] and iou(pred[:4], gt[:4]) >= iou_thresholds[0]:
            TP[pred[5]] += 1
            IoUs[pred[5]].append(iou(pred[:4], gt[:4]))  # Store the IoU for this class
            matched.add(i)
            matched_flag = True
            break
    if not matched_flag:
        FP[pred[5]] += 1

# Calculate FN for each class
for i, gt in enumerate(gt_boxes):
    if i not in matched:
        FN[gt[4]] += 1

# Calculate precision, recall, and IoU for each class and overall
precision = {cls: TP[cls] / (TP[cls] + FP[cls]) if (TP[cls] + FP[cls]) > 0 else 0 for cls in range(len(class_names))}
recall = {cls: TP[cls] / (TP[cls] + FN[cls]) if (TP[cls] + FN[cls]) > 0 else 0 for cls in range(len(class_names))}
IoU = {cls: np.mean(IoUs[cls]) if IoUs[cls] else 0 for cls in range(len(class_names))}

# Overall precision and recall
total_TP = sum(TP.values())
total_FP = sum(FP.values())
total_FN = sum(FN.values())

overall_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
overall_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0

# === STEP 7: Calculate mAP@0.5 and mAP@0.5:0.95 ===
def calc_map(pred_boxes, gt_boxes, iou_thresholds):
    ap_per_class = {}
    for cls in range(len(class_names)):
        # Placeholder for calculating AP (using IoU thresholds)
        ap_per_class[cls] = {'mAP50': 0, 'mAP50-95': 0}
        # In practice, you would implement AP calculation using the precision-recall curve at different IoU thresholds.
    return ap_per_class

ap_per_class = calc_map(pred_boxes, gt_boxes, iou_thresholds)

# === PRINT EVALUATION RESULTS ===
print("\n=== 📊 YOLOv12 Evaluation (Single Image) ===")
print(f"Overall Precision: {overall_precision:.3f}")
print(f"Overall Recall: {overall_recall:.3f}")
print(f"Overall IoU: {np.mean([IoU[cls] for cls in IoU]):.3f}")

# Print FP, TP, FN for each class along with precision, recall, IoU
for cls, name in enumerate(class_names):
    print(f"\nClass: {name}")
    print(f"  True Positives (TP): {TP[cls]}")
    print(f"  False Positives (FP): {FP[cls]}")
    print(f"  False Negatives (FN): {FN[cls]}")
    print(f"  Precision: {precision[cls]:.3f}")
    print(f"  Recall: {recall[cls]:.3f}")
    print(f"  IoU: {IoU[cls]:.3f}")
    print(f"  mAP50: {ap_per_class[cls]['mAP50']:.3f}")
    print(f"  mAP50-95: {ap_per_class[cls]['mAP50-95']:.3f}")

# === STEP 8: Visualize Results ===
img = cv2.imread(image_path)

# Visualize Ground Truth Boxes (Green) and Predicted Boxes (Blue)
for x1, y1, x2, y2, cls in gt_boxes:
    cls = int(cls)  # Ensure that cls is an integer
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(img, f"GT {class_names[cls]}", (int(x1), int(y1)-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

for x1, y1, x2, y2, conf, cls in pred_boxes:
    cls = int(cls)  # Ensure that cls is an integer
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
    cv2.putText(img, f"Pred {class_names[cls]} {conf:.2f}", (int(x1), int(y1)-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

# Show the image with bounding boxes
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Predictions (Blue) vs GT (Green)")
plt.axis('off')
plt.show()
